In [1]:
import pandas as pd
import datetime as dt
import os

In [2]:
data_dir = './data/'
today = dt.date.today()

In [3]:
metabolomics_data_scaled = pd.read_csv(os.path.join(data_dir, 'autoscaled_lung_2020-06-23.csv'), header = [0,1])

sample_types = ['Normal', '4 Hr Sham', '4 Hr AKI', '24 Hr Sham', '24 Hr AKI', '7 Day Sham', '7 Day AKI']

In [4]:
metabolomics_data_scaled.head()

compound            Pathway    Normal                      \
  Unnamed: 0_level_1 Unnamed: 1_level_1       523       526       529   
0            Alanine        Amino acids -0.336500  0.229450 -0.153261   
1           Arginine        Amino acids -0.429142 -0.385544 -0.368202   
2         Asparagine        Amino acids -1.033009 -0.655870 -0.982483   
3          Aspartate        Amino acids  0.003941 -0.158761 -0.056675   
4           Cysteine        Amino acids -0.773488 -0.972479 -0.850761   

                                                     ... 7 Day Sham            \
        532       535       547       550       553  ...        502       504   
0  1.028644 -0.874709  2.024181 -0.185609 -0.285072  ...   1.960299 -0.407205   
1 -0.009323 -0.442200  1.513968 -0.355217 -0.016899  ...   0.463069 -0.415095   
2  0.026174 -1.207261  0.399279 -0.713444 -0.828145  ...   0.367635 -0.792755   
3  0.372621 -0.672211  2.170162  0.493237 -0.008459  ...   1.791246  0.532116   
4  0.553739 -0.998322  1.022017 -0.421950 -0.263286  ...   1.215435 -0.266377   

                      7 Day AKI                                          \
        508       510       496       498       500       501       503   
0 -0.972669  1.391121 -0.021510  0.059576 -0.040353 -0.858209 -1.295198   
1 -1.088772 -0.413004 -0.723250 -1.108063 -0.751130 -1.168714 -1.044329   
2 -1.301692 -0.047250 -0.603651 -0.622078 -0.759393 -1.038891 -0.907662   
3 -0.348095  2.329130  2.469424  0.929382  1.600209  0.551191 -0.173145   
4 -1.005669  1.764646  0.546711 -0.216357 -0.430195 -0.467082 -0.698897   

             
        505  
0  0.012093  
1 -0.380884  
2 -0.536896  
3  1.629194  
4  0.032793  

[5 rows x 67 columns]

In [16]:
from scipy import stats
metabolomics_data_scaled["anova_pvalue"] = metabolomics_data_scaled[sample_types].apply(lambda x: stats.f_oneway(*[x[sample].values.tolist() for sample in sample_types])[1], axis = 1)

In [6]:
metabolomics_data_scaled.rename(columns = {'Unnamed: 0_level_1':'', 'Unnamed: 1_level_1':''}, level = 1, inplace = True)

In [38]:
# output anova significant metabolites
metabolomics_data_scaled.loc[metabolomics_data_scaled.anova_pvalue < 0.05, ['compound','Pathway'] + sample_types].\
    to_csv(os.path.join(data_dir, 'autoscaled_lung_anovasig_{}.csv'.format(today)), index = False)

## Post Hoc Tests

In [7]:
metabolomics_data_scaled.head()

compound      Pathway    Normal                                          \
                                 523       526       529       532       535   
0     Alanine  Amino acids -0.336500  0.229450 -0.153261  1.028644 -0.874709   
1    Arginine  Amino acids -0.429142 -0.385544 -0.368202 -0.009323 -0.442200   
2  Asparagine  Amino acids -1.033009 -0.655870 -0.982483  0.026174 -1.207261   
3   Aspartate  Amino acids  0.003941 -0.158761 -0.056675  0.372621 -0.672211   
4    Cysteine  Amino acids -0.773488 -0.972479 -0.850761  0.553739 -0.998322   

                                 ... 7 Day Sham                     7 Day AKI  \
        547       550       553  ...        504       508       510       496   
0  2.024181 -0.185609 -0.285072  ...  -0.407205 -0.972669  1.391121 -0.021510   
1  1.513968 -0.355217 -0.016899  ...  -0.415095 -1.088772 -0.413004 -0.723250   
2  0.399279 -0.713444 -0.828145  ...  -0.792755 -1.301692 -0.047250 -0.603651   
3  2.170162  0.493237 -0.008459  ...   0.532116 -0.348095  2.329130  2.469424   
4  1.022017 -0.421950 -0.263286  ...  -0.266377 -1.005669  1.764646  0.546711   

                                                    anova_pvalue  
        498       500       501       503       505               
0  0.059576 -0.040353 -0.858209 -1.295198  0.012093     0.003292  
1 -1.108063 -0.751130 -1.168714 -1.044329 -0.380884     0.006425  
2 -0.622078 -0.759393 -1.038891 -0.907662 -0.536896     0.000541  
3  0.929382  1.600209  0.551191 -0.173145  1.629194     0.000002  
4 -0.216357 -0.430195 -0.467082 -0.698897  0.032793     0.073168  

[5 rows x 68 columns]

In [8]:
def ttest(metabolite_row, metabolite_name, relv_combos):
    means = [[metabolite_row[c1].mean(),metabolite_row[c2].mean()] for c1, c2 in relv_combos]
    results = [stats.ttest_ind(metabolite_row[c1],metabolite_row[c2]) for c1, c2 in relv_combos]
    df = pd.DataFrame(relv_combos, columns = ['g1','g2']).join(pd.DataFrame(means, columns = ['g1_mean', 'g2_mean'])).\
        join(pd.DataFrame([i.pvalue for i in results], columns = ['p_value']))
    df['compound'] = metabolite_name.values[0]
    return df

g1 = ["Normal", "Normal", "Normal", "Normal", "Normal", "Normal", "4 Hr Sham", "24 Hr Sham", "7 Day Sham"]
g2 = ["4 Hr Sham", "4 Hr AKI", "24 Hr Sham", "24 Hr AKI", "7 Day Sham", "7 Day AKI", "4 Hr AKI", "24 Hr AKI", "7 Day AKI"]
relv_combos = [[c1,c2] for c1,c2 in zip(g1,g2)]

In [17]:
from statsmodels.stats.multitest import multipletests
def multi_test_adjust(p_vals, method = 'fdr_by'):
    return multipletests(p_vals, method = method, is_sorted = False)[1].tolist()

Test to remove outliers

In [29]:
outliers = ['547','544','503']
metabolomics_data_scaled_no_outliers = metabolomics_data_scaled.drop(columns = outliers, level = 1)

In [30]:
mtt_values_no_outliers = metabolomics_data_scaled_no_outliers.apply(lambda x: ttest(x[sample_types], x["compound"], relv_combos), axis = 1)
mtt_metabolite_df_no_outliers = pd.concat(mtt_values_no_outliers.values.tolist())

In [31]:
mtt_metabolite_df_no_outliers['p_value_adjusted_sidak_outliers_removed'] = mtt_metabolite_df_no_outliers.groupby('compound')['p_value'].\
        transform(lambda x: multi_test_adjust(x.values.tolist(), method = 'sidak'))

With Outliers

In [32]:
mtt_values = metabolomics_data_scaled.apply(lambda x: ttest(x[sample_types], x["compound"], relv_combos), axis = 1)
mtt_metabolite_df = pd.concat(mtt_values.values.tolist())

In [34]:
# # p value correction
# from statsmodels.stats.multitest import multipletests
# methods = ['bonferroni', 'sidak','holm-sidak','simes-hochberg','hommel','fdr_bh','fdr_by','fdr_tsbh','fdr_tsbky']
# for method in methods:
#     mtt_metabolite_df['p_value_adjusted_{}'.format(method)] = multipletests(mtt_metabolite_df['p_value'], method = method, is_sorted = False)[1].tolist()

In [35]:
mtt_metabolite_df['p_value_adjusted_sidak_all_samples'] = mtt_metabolite_df.groupby('compound')['p_value'].\
        transform(lambda x: multi_test_adjust(x.values.tolist(), method = 'sidak'))

mtt_metabolite_df['p_value_adjusted_fdr_by_all_samples'] = mtt_metabolite_df.groupby('compound')['p_value'].\
        transform(lambda x: multi_test_adjust(x.values.tolist(), method = 'fdr_by'))

In [36]:
mtt_metabolite_df

,g1,g2,g1_mean,g2_mean,p_value,compound,p_value_adjusted_sidak_all_samples,p_value_adjusted_fdr_by_all_samples
0,Normal,4 Hr Sham,-0.015327,0.477393,0.187033,Alanine,0.844884,0.793665
1,Normal,4 Hr AKI,-0.015327,-0.578707,0.150321,Alanine,0.769169,0.765456
2,Normal,24 Hr Sham,-0.015327,0.823642,0.112736,Alanine,0.659218,0.717584
3,Normal,24 Hr AKI,-0.015327,-0.659393,0.095128,Alanine,0.593290,0.717584
4,Normal,7 Day Sham,-0.015327,0.185279,0.652836,Alanine,0.999927,1.000000
...,...,...,...,...,...,...,...,...
4,Normal,7 Day Sham,0.069923,-1.020494,0.000321,2-Oxo-7-methylthioheptanoic acid,0.002887,0.004089
5,Normal,7 Day AKI,0.069923,-1.043773,0.000909,2-Oxo-7-methylthioheptanoic acid,0.008154,0.007717
6,4 Hr Sham,4 Hr AKI,0.687140,0.887759,0.561009,2-Oxo-7-methylthioheptanoic acid,0.999395,1.000000
7,24 Hr Sham,24 Hr AKI,0.677109,-0.777222,0.000252,2-Oxo-7-methylthioheptanoic acid,0.002262,0.004089


In [44]:
mtt_metabolite_df.rename(columns = {'p_value':'p_value_no_adjustment_all_samples'}, inplace = True)

mtt_metabolite_df_no_outliers.rename(columns = {'p_value':'p_value_no_adjustment_outliers_removed'}, inplace = True)

In [49]:
s = mtt_metabolite_df_no_outliers[['g1','g2', 'compound', 'p_value_no_adjustment_outliers_removed','p_value_adjusted_sidak_outliers_removed']].copy()
f = mtt_metabolite_df[['g1','g2','p_value_no_adjustment_all_samples', 'compound', 'p_value_adjusted_sidak_all_samples', 'p_value_adjusted_fdr_by_all_samples']].copy()

In [51]:
final = s.merge(f, on = ['g1','g2','compound'], how = 'outer')

In [54]:
final.to_csv('./outliers_and_all_samples_mtt_table.csv', index = False)

In [46]:
# for method in methods:
#     print(method, sum(multipletests(mtt_metabolite_df['p_value'], method = method, is_sorted = False)[1] <= 0.05))
#     #print("Num Unique Metabolites Significant: ", mtt_metabolite_df.compound.nunique())

In [47]:
mtt_metabolite_df.compound.nunique()

132

In [48]:
# add columns indicated increase or decrease and signnificance
mtt_metabolite_df.loc[mtt_metabolite_df.g2_mean > mtt_metabolite_df.g1_mean, "is_increase"] = True
mtt_metabolite_df.loc[mtt_metabolite_df.g2_mean < mtt_metabolite_df.g1_mean, "is_increase"] = False

mtt_metabolite_df.loc[mtt_metabolite_df.p_value_adjusted < 0.05, "significant"] = True
mtt_metabolite_df.loc[mtt_metabolite_df.p_value_adjusted >= 0.05, "significant"] = False

In [50]:
mtt_metabolite_df.significant.value_counts()

False    1036
True      152
Name: significant, dtype: int64

In [53]:
mtt_metabolite_df.to_csv(os.path.join(data_dir, 'lung_mt_significance_table_{}.csv'.format(today)), index = False)

In [14]:
## should there be logic for such specific cases? 

In [171]:
## sig sham to normal
group_1 = 'Normal'
group_2 = ['4 Hr Sham', '24 Hr Sham', '7 Day Sham']
sig_2_sham = mtt_metabolite_df.loc[(mtt_metabolite_df.p_value_corrected <= 0.05) & \
                      (mtt_metabolite_df.g1 == group_1) & (mtt_metabolite_df.g2.isin(group_2))].sort_values(by = ['g1','g2'])

In [172]:
## aki to normal
group_1 = 'Normal'
group_2 = ['4 Hr AKI', '24 Hr AKI', '7 Day AKI']
aki_to_normal = mtt_metabolite_df.loc[(mtt_metabolite_df.g1 == group_1) & (mtt_metabolite_df.g2.isin(group_2))].sort_values(by = ['g1','g2'])

In [173]:
## aki to sham
group_1 = ['4 Hr Sham', '24 Hr Sham', '7 Day Sham']
group_2 = ['4 Hr AKI', '24 Hr AKI', '7 Day AKI']
aki_to_sham = mtt_metabolite_df.loc[(mtt_metabolite_df.g1.isin(group_1)) & (mtt_metabolite_df.g2.isin(group_2))].sort_values(by = ['g1','g2'])

In [179]:
# sig to aki
relv_cols = ['compound','g1','g2','is_increase','p_value_corrected']
aki_comp_df = aki_to_sham[relv_cols].merge(aki_to_normal[relv_cols], on = ['compound', 'g2'], how = 'inner', suffixes=['_c2_normal','_c2_sham'])

In [180]:
def sig2aki():
    if is_increase_c2_normal and is_increase_c2_sham and p_value_corrected_c2_sham <= 0.05 and p_value_corrected_c2_normal <= 0.05:
        return "Sig2AKI"
    elif is_increase_c2_normal and not is_increase_c2_sham and p_value_corrected_c2_sham <= 0.05:
        
aki_comp_df

,compound,g1_c2_normal,g2,is_increase_c2_normal,p_value_corrected_c2_normal,g1_c2_sham,is_increase_c2_sham,p_value_corrected_c2_sham
0,Alanine,24 Hr Sham,24 Hr AKI,False,0.303652,Normal,False,1.000000
1,Arginine,24 Hr Sham,24 Hr AKI,False,1.000000,Normal,True,1.000000
2,Asparagine,24 Hr Sham,24 Hr AKI,True,1.000000,Normal,True,0.515579
3,Aspartate,24 Hr Sham,24 Hr AKI,False,0.253149,Normal,False,0.379008
4,Cysteine,24 Hr Sham,24 Hr AKI,False,1.000000,Normal,True,1.000000
...,...,...,...,...,...,...,...,...
391,N-Amidino-L-aspartate,7 Day Sham,7 Day AKI,True,0.082177,Normal,True,0.038269
392,Hydroxyacetone phosphate,7 Day Sham,7 Day AKI,False,1.000000,Normal,True,1.000000
393,trans-Homoaconitate,7 Day Sham,7 Day AKI,False,1.000000,Normal,False,1.000000
394,N-Carbamyl-L-glutamate,7 Day Sham,7 Day AKI,False,1.000000,Normal,True,1.000000


In [ ]:
get_sig_mets <- function(p_values, SHAM, AKI, comparison_cat, i) {
    # Significant to AKI
    increased = NA
    decreased = NA
    if(comparison_cat == 'Sig2AKI') {
    # Increased AKI
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == AKI), ]$p.value <= 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 & 
                p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value > 0.05 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) < 0) {increased = names(p_values[i])}
                    # this is a factor, need to change to int, using the t.test value to infer direction inc vs dec
    # decreased AKI
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == AKI), ]$p.value <= 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 & 
                p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value > 0.05 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) > 0) {decreased = names(p_values[i])}
    }
    
    # more in AKI
    if(comparison_cat == 'MoreInAKI') {
    # Increased More in 4 hr AKI
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 & 
                as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$t.value)) < 0 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) < 0) {increased = names(p_values[i])}
    # decreased more in 4 hr AKI
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 & 
                as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$t.value)) > 0 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) > 0) {decreased = names(p_values[i])}
    }
    
     # more in AKI
    if(comparison_cat == 'OppInAKI') {
    # increased opposite
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 &
                p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == AKI), ]$p.value <= 0.05 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$t.value)) > 0 &
                        as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) < 0) {increased = names(p_values[i])}
    # decreased opposite
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 &
                p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == AKI), ]$p.value <= 0.05 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$t.value)) < 0 &
                        as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) > 0) {decreased = names(p_values[i])}
    }
    
    if(comparison_cat == 'NoEfAKI') {
    # No effect in AKI
    # increase noef AKI
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == AKI), ]$p.value > 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 & 
                p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) > 0) {increased = names(p_values[i])}
    # decrease  no effect AKI
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == AKI), ]$p.value > 0.05 & 
            p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$p.value <= 0.05 & 
                p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 &
                    as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == SHAM & var2 == AKI), ]$t.value)) < 0) {decreased = names(p_values[i])}
    }
    
    if(comparison_cat == 'Sig2Sham') {
    # increase sham
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 & 
            as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$t.value)) < 0) {increased = names(p_values[i])}
    #decrease sham
    if (p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$p.value <= 0.05 & 
            as.numeric(as.character(p_values[[i]][with(p_values[[i]], var1 == 'Normal' & var2 == SHAM), ]$t.value)) > 0) {decreased = names(p_values[i])}
    }
    return (list(increased, decreased))
}